In [1]:
from lean_interact import Command, LeanServer, LeanREPLConfig, TempRequireProject, LocalProject
localprojectdir = "../matheye/benchmarks/"
config = LeanREPLConfig(verbose=True, project=LocalProject(localprojectdir))
print(config.__dict__)

server = LeanServer(config)


Build completed successfully.


No files to download
Decompressing 6641 file(s)
Unpacked in 90 ms
Completed successfully!
Build completed successfully.
{'lean_version': 'v4.19.0', 'project': LocalProject(directory='../matheye/benchmarks/', build=True), 'repl_git': 'https://github.com/augustepoiroux/repl', 'repl_rev': 'v1.0.8', 'cache_dir': '/home/matheye/anaconda3/lib/python3.12/site-packages/lean_interact/cache', 'memory_hard_limit_mb': None, 'verbose': True, '_stdout': None, '_stderr': None, 'repo_name': 'augustepoiroux/repl', 'cache_clean_repl_dir': '/home/matheye/anaconda3/lib/python3.12/site-packages/lean_interact/cache/augustepoiroux/repl/repl_clean_copy', '_cache_repl_dir': '/home/matheye/anaconda3/lib/python3.12/site-packages/lean_interact/cache/augustepoiroux/repl/repl_v1.0.8_v4.19.0', '_working_dir': '../matheye/benchmarks/'}


In [3]:

response = server.run(Command(cmd=
"""import Mathlib
theorem induction_with_have (n : ℕ) : n * (n + 1) / 2 + (n + 1) = (n + 1) * (n + 2) / 2 := by
  induction' n with k ih
  · -- base case n = 0
    simp
    have h1 : (0 : ℕ) * 1 / 2 = 0 := by norm_num
    have h2 : h1.trans rfl = (0 + 1) := by simp
    norm_num
  · -- inductive step
    simp [Nat.succ_eq_add_one]
    have h1 : k * (k + 1) / 2 + (k + 1) = (k + 1) * (k + 2) / 2 := by
      exact ih
    have h2 : (k + 1) * (k + 2) / 2 + (k + 2) = (k + 2) * (k + 3) / 2 := by
      have h3 : (k + 1) * (k + 2) = k * (k + 1) + 2 * (k + 1) := by ring
      have h4 : (k + 2) * (k + 3) = (k + 1) * (k + 2) + 2 * (k + 2) := by ring
      have h5 : 2 * (k + 2) = (k + 2) + (k + 2) := by ring
      ring
    exact h2
"""))
print(response)


CommandResponse(env=1, messages=[Message(severity='error', data='no goals to be solved', end_pos=Pos(line=6, column=8), start_pos=Pos(line=6, column=4)), Message(severity='error', data='simp made no progress', end_pos=Pos(line=10, column=30), start_pos=Pos(line=10, column=4))])


In [1]:
from lean_interact import Command, LeanServer, LeanREPLConfig, TempRequireProject, LocalProject, ProofStep
localprojectdir = "../matheye/benchmarks/"
config = LeanREPLConfig(verbose=True, project=LocalProject(localprojectdir))
print(config.__dict__)

server = LeanServer(config)
response = server.run(Command(cmd="""import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat
example (a b c d e : ℝ) (h : a = b + c) (h' : b = d - e) : a + e = d + c := by
  rw [h]
  sorry
"""))
print(response.sorries)
response = server.run(ProofStep(proof_state=0, tactic="""(
  rw [h']
  ring
)"""))
print(response)

Build completed successfully.
No files to download
Decompressing 6641 file(s)
Unpacked in 60 ms
Completed successfully!
Build completed successfully.
{'lean_version': 'v4.19.0', 'project': LocalProject(directory='../matheye/benchmarks/', build=True), 'repl_git': 'https://github.com/augustepoiroux/repl', 'repl_rev': 'v1.0.8', 'cache_dir': '/home/matheye/anaconda3/lib/python3.12/site-packages/lean_interact/cache', 'memory_hard_limit_mb': None, 'verbose': True, '_stdout': None, '_stderr': None, 'repo_name': 'augustepoiroux/repl', 'cache_clean_repl_dir': '/home/matheye/anaconda3/lib/python3.12/site-packages/lean_interact/cache/augustepoiroux/repl/repl_clean_copy', '_cache_repl_dir': '/home/matheye/anaconda3/lib/python3.12/site-packages/lean_interact/cache/augustepoiroux/repl/repl_v1.0.8_v4.19.0', '_working_dir': '../matheye/benchmarks/'}
[Sorry(proof_state=0, end_pos=Pos(column=7, line=9), goal="a b c d e : ℝ\nh : a = b + c\nh' : b = d - e\n⊢ b + c + e = d + c", start_pos=Pos(column=2, lin

In [ ]:
response = server.run(ProofStep(proof_state=0, tactic="rw [h']"))

ProofStepResponse(proof_status='Completed', goals=[], proof_state=1)


In [21]:
from lean_interact import Command, LeanServer, LeanREPLConfig, TempRequireProject, LocalProject
localprojectdir = "../matheye/benchmarks/"
config = LeanREPLConfig(verbose=True, project=LocalProject(localprojectdir))
print(config.__dict__)

server = LeanServer(config)
response = server.run(Command(cmd="""import Mathlib
theorem eq_comm_demo (x y : ℕ) : x = y ↔ y = x := by
  constructor
  · 
    have a: x = y -> y = x := by
      intro h 
      rw [<- h]
      rw [h]
      <;>linarith
    exact a
  · 
    rw [a]
    rw [h]

""", all_tactics=True))

for tactic in response.tactics:
    print(tactic.tactic + " " + str(tactic.start_pos) + " " + str(tactic.end_pos))

constructor Pos(line=3, column=2) Pos(line=3, column=13)
have a : x = y -> y = x := by
  intro h
  rw [<- h]
  rw [h] <;> linarith Pos(line=5, column=4) Pos(line=9, column=17)
intro h Pos(line=6, column=6) Pos(line=6, column=13)
rw [<- h] Pos(line=7, column=6) Pos(line=7, column=15)
exact a Pos(line=10, column=4) Pos(line=10, column=11)
rw [a] Pos(line=12, column=4) Pos(line=12, column=10)


In [27]:
from lean_interact import ProofStep


In [28]:
response = server.run(ProofStep(proof_state=1, tactic="""(
nlinarith
)"""))
print(response)

ProofStepResponse(proof_status="Error: kernel type check failed: (kernel) declaration has metavariables '[anonymous]'", goals=[], proof_state=3)
